In [ ]:
import os
import sys
import logging
sys.path.append("../")
%load_ext autoreload

os.listdir("..\data")
path = "..\data"
targets = []
for folder in os.listdir(path):
    for f in os.listdir(os.path.join(path, folder)):
        #if f.endswith(".png") or f.endswith(".jpg"):
        if f.endswith(".pdf"):
            targets.append(os.path.join(path, folder, f))
print(targets)

In [ ]:
%autoreload 2

from data_loaders.textract_image_loader import TextractImageLoader
from data_loaders.pdf_loader import PDFLoader

from outputs.default_writer import DefaultWriter
from outputs.pluto_writer import PlutoWriter

from extractor import StatblockExtractor
from utils.config import get_config, get_argparser
from utils.logger import get_logger

from fifthedition.creature_printer import pretty_format_creature

# Get arguments
parser = get_argparser()
args = parser.parse_args([
    "..\data\clockwork_drake\clockwork_drake.jpg", 
    "--config", r"..\default.conf", 
    "--cache",  r"..\.cache",
    "--no-cache"
])
print(args)

# Get config file
config = get_config(args)

logger = get_logger(args.debug, args.logs)

se = StatblockExtractor(config, logger)
se.register_data_loader(TextractImageLoader)
se.register_data_loader(PDFLoader)

se.register_output_writer(DefaultWriter, append=True)
se.register_output_writer(PlutoWriter, append=True)

#args.output = "../images.json"
#args.output_format = "5et"

logger.setLevel(logging.DEBUG)

se.select_writer(args.output_format)

targets = [r"..\data\pdfs\Guildmasters Guide to Ravnica.pdf"]

for target in targets:
    print(target)
    logger.info("Loading creatures from {}".format(target))
    
    results = se.parse(target, args.output, pages=[3])
    parsed_statblocks, statblocks = results

    num_pages = len(statblocks.keys())
    logger.info("Found {} page{} of statblocks".format(num_pages, 's' if num_pages > 1 else ''))
    
    for page in parsed_statblocks:
        print("Page {}:".format(page))
        for creature in parsed_statblocks[page]:
            print()
            print(pretty_format_creature(creature))
            print()